# Explaining the loss of a model

## Introduction

In theory, SHAP values can be used to explain anything about a machine learning model that can be formulated as the "value of the game" $v(S)$ for a coalition $S$.

Typically, when people talk about using shap explanations on a machine learning model, they refer to explaining the contribution to the prediction output.

There are however many alternative applications in the machine learning literature, including for example **feature selection**, where $v(S)$ is the model performance, **data valuation**, where $v(S)$ is the goodness of fit on test data, or **monitoring**, where $v(S)$ is the model loss.


### Motivation

In this notebook, I will look closer at how shap values can be used to explain the loss of a model. 

In particular, I will be interested in explaining the contribution of each feature/covariate to the MSE at the leaf level of a tree ensemble regression model. This will allow to characterize the hetereogeneity of CATEs within the leafs of the tree ensemble.

### Existing approaches and shortcomings

In the existing implementation of [treeSHAP](https://shap-lrjball.readthedocs.io/en/latest/generated/shap.TreeExplainer.html) in the mainstream package for shap values, `shap`, there is the possibility to compute the `log_loss` of binary classification models, but not more general loss functions. (Note that when computing any loss, one has to pass the label to the shap explainer as well.)

It also mentions that it is possible to pass "the name of a supported prediction method on the model object" for which the shap values will then be computed. Besides raw output and probabilities for classification, however, there are no default methods to get the MSE.

Instead of having to modify the `shap` package, there might be an easy way by creating a wrapper class for our RandomForestRegressor that exhibits a method that returns the leaf node MSE for any observation.

A complication is that this "custom" model output shap explanation is only supported for `feature_perturbation="interventional`, as opposed to the default `feature_perturbation="tree_path_dependent"`, which uses the background training samples that went down each tree path to compute "observational conditional expectations" from which to sample "inactive" features. 

The `interventional` method requires to pass a background dataset (with regards to which runtime scales linearly), and the documentation suggests to use anywhere from 100 to 1000 random background samples. It claims that the `interventional` approach *"breaks the dependencies between features according to the rules dictated by causal inference [(Janzing et al. 2019)](https://proceedings.neurips.cc/paper/2019/file/2172fde49301047270b2897085e4319d-Paper.pdf)"*. It is not yet clear to me how this can be done without integrating any causal knowledge about our Xs into the model - the formulation is probably unfortunate at best. 

The most convincing way I have seen this done is as demonstrated by (Heskes, Bucur, Sijben and Claassen 2020) from Radboud University, which suggest and demonstrate to use Pearls do-calculus to compute interventional conditional distributions, and so naturally asks for a DAG.

In any case, this is important to be aware of. I think one can make a case for being interested in the interpretation that the `interventional` approach, but if not, then there is no way around touching the `shap` package (which might be very difficult).

### Idea behind explaining loss

Let $\Pi$ denote the partition of a fit regression tree, and $l(x, \Pi)$ the leaf node that $x$ falls into under partition $\Pi$.

For each input vector $x_i$, a standard fitted decision tree regressor provides output 
$$
f(x_i) = \frac{1}{\#\{k \in l(x_i, \Pi)\}} \sum_{k \in l(x_i, \Pi)}\bar{Y_k}
$$

Similarly, for each input vector $x$, we can consider the loss at each input vector:

$$
\begin{align*}
\text{loss}(x_i) &= (y-f(x_i))^2 \\
&= (y - \frac{1}{\#\{k \in l(x_i, \Pi)\}} \sum_{k \in l(x_i, \Pi)}\bar{Y_k})^2
\end{align*}
$$

Naturally, the overall MSE loss of the data is just the average of the loss at each input vector:
$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^n \text{loss}(x_i)
$$

These are feasible to compute for any input vector $x_i$ and any partition $\Pi$ since we know the outcome $y_i$ for each collection of covariates in an input vector $x_i$. This will not be the case for CATE in causal trees, for the standard "fundamental problem of causal inference". In that case we will resort to corresponding estimators from our estimation data (Athey & Wager, PNAS 2016).

Let's now see how we can use shap to explain such loss.

## Defining a custom tree regressor

I will begin with an exposition on a single tree.

In [2]:
# imports
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.11 from "/Users/lasse/miniconda3/envs/causal_trees/bin/python"
  * The NumPy version is: "1.23.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/lasse/miniconda3/envs/causal_trees/lib/python3.11/site-packages/numpy/core/_multiarray_umath.cpython-311-darwin.so, 0x0002): Library not loaded: @rpath/libcblas.3.dylib
  Referenced from: <0CB3976A-DD5B-39DE-BAC5-4D0294F9A6F4> /Users/lasse/miniconda3/envs/

In [ ]:
from sklearn.datasets import load_diabetes

# Load the Diabetes dataset
diabetes = load_diabetes()

# create joint dataframe
df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target

df.shape


(442, 11)

: 

: 

In [ ]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


: 

: 

In [ ]:
# define a new estimator with our new custom loss

class CustomDecisionTree(DecisionTreeRegressor):
    def __init__(
        self,
        *,
        criterion="squared_error",
        splitter="best",
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0,
        max_features=None,
        random_state=None,
        max_leaf_nodes=None,
        min_impurity_decrease=0.0,
        ccp_alpha=0.0,
    ):
        super().__init__(
            criterion=criterion,
            splitter=splitter,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            min_weight_fraction_leaf=min_weight_fraction_leaf,
            max_features=max_features,
            max_leaf_nodes=max_leaf_nodes,
            random_state=random_state,
            min_impurity_decrease=min_impurity_decrease,
            ccp_alpha=ccp_alpha,
        ) # this respects the scikit-learn demanded init structure
    
    def leaf_mean_loss(self, X, y):
        """
        Computes the squared loss between the predicted values and the mean value of the leaf nodes.

        Parameters:
            X (array-like or sparse matrix): Input samples.

        Returns:
            array-like: Array of squared errors between the predicted values and the mean value of the leaf nodes.
        """
        # check that tree is fitted
        assert self.tree_ is not None

        # return squared loss
        sq_loss = (y - self.predict(X))**2
        
        return sq_loss




: 

: 

Let's try to train our custom tree on the diabetes dataset.

In [ ]:
# instantiate GridSearchCV
grid = GridSearchCV(
    estimator=CustomDecisionTree(),
    param_grid={'max_depth': np.arange(2, 10)},
    cv = 4)
grid.fit(df.drop("target",axis = 1), df.target)

# get the best estimator
best_estimator = grid.best_estimator_

# get the best parameters
grid.best_params_

{'max_depth': 2}

: 

: 

In [ ]:
# compute the loss for each observation
loss = best_estimator.leaf_mean_loss(df.drop("target",axis = 1), df.target)

: 

: 

In [ ]:
# lets check this adds up to the MSE
print(np.sum(loss) / len(loss))
print(mean_squared_error(df.target, best_estimator.predict(df.drop("target",axis = 1))))

3360.050096675736
3360.050096675736


: 

: 

Having now defined a custom loss function, we should be able to use default shap code to explain the loss of our model.

### SHAP explanations

(NOTE: Deprecations in numpy 1.23 -> 1.24 have caused some issues in the shap package, see [here](https://github.com/shap/shap/issues/2911). It is best to go with numpy 1.23 and shap 0.42)

Let's first compute the standard "prediction" shap explanation on our diabetes data using our custom tree regressor.

In [ ]:
import shap
print(shap.__version__)

# compute SHAP values
explainer = shap.TreeExplainer(best_estimator)
prediction_shap_values = explainer.shap_values(df.drop("target",axis = 1))

/Users/lasse/miniconda3/envs/causal_trees/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/lasse/miniconda3/envs/causal_trees/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def

: 

: 

: 

: 